In [23]:
import pandas as pd
import numpy as np

In [44]:
df = pd.read_csv('raw_logs.csv',header=0).groupby('session')
for name, group in df:
    print('name = {0}, group = {1}'.format(name, group.size))
    
# Sort timestamp inside a session and get the final average MPG for every trip (kff1205)

name = 1509673999804, group = 45306
name = 1509715520133, group = 13311
name = 1509814879698, group = 12636
name = 1510237809601, group = 63747
